[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/artbert/VoiceChatLLM/blob/main/Voice_Chat_With_Bielik_Colab.ipynb)

![Runtime: Colab](https://img.shields.io/badge/runtime-Colab-4285F4?style=for-the-badge&logo=googlecolab&logoColor=white)
![GPU required](https://img.shields.io/badge/GPU-Required-red?style=for-the-badge&logo=nvidia&logoColor=white)

<div>
<center>
<img src="https://raw.githubusercontent.com/artbert/VoiceChatLLM/refs/heads/main/assets/voice_chat_with_bielik_colab_demo.png" alt = "Bielik Demo" width="640"/>
</center>
</div>


## Instrukcja konfiguracji

Zanim odpalimy nasz czat głosowy z LLM, szybko przygotuj środowisko – instalacja bibliotek i modeli zajmuje dosłownie chwilę. Pobieranie Bielika może jednak zająć trochę dłużej.

### Wymagania wstępne

- Środowisko Google Colab  
- Python 3

### Instalacja bibliotek

Uruchom komórkę z `pip install`, by pobrać potrzebne pakiety: piper, ffmpeg, vosk, accelerate i bitsandbytes.

### Pobieranie modeli

Ta aplikacja opiera się na trzech modelach:  
- Piper (Text-to-Speech)  
- Vosk (`vosk-model-small-pl-0.22`) do rozpoznawania mowy  
- Bielik (`speakleash/Bielik-11B-v2.6-Instruct`) jako nasz LLM  

Vosk pobierze się automatycznie przy pierwszym wywołaniu `vosk.Model`. Bielik zostanie ściągnięty i załadowany przez `transformers`. Potrzebujesz karty graficznej, żeby LLM działał płynnie.

### Uruchamianie aplikacji

Gdy wszystkie biblioteki i modele są gotowe, odpal pozostałe komórki w notatniku krok po kroku, żeby uruchomić interfejs czatu i od razu go przetestować.

## Instrukcja użytkowania

Tu zaczyna się zabawa – sprawdź, jak działa czat głosowy z LLM.

### Uruchamianie aplikacji

Znajdź komórkę zaraz po nagłówku „Uruchamianie aplikacji” i ją wykonaj. Interfejs czatu pojawi się od razu.

### Interakcja z aplikacją

Po starcie zobaczysz prosty panel czatu.

#### Wprowadzanie wiadomości użytkownika

Masz dwie opcje:

- Wejście głosowe: kliknij ikonę mikrofonu, mów wyraźnie, kliknij STOP, by zakończyć nagranie. Transkrypcja pojawi się w polu tekstowym. Przed wysłaniem możesz wprowadzić korektę tekstu.
- Wejście tekstowe: wpisz wiadomość i naciśnij Enter lub kliknij przycisk wysyłania.

Odpowiedź od LLM pojawi się jako tekst, a jeśli TTS działa poprawnie, usłyszysz ją również na głos.

#### Sterowanie czatem

Asystent "pamięta" całość dotychczasowej konwersacji (dokładnie to, co widać w oknie) i jest to ograniczone przez szerokość okna kontekstowego wybranego modelu językowego.<br>Aby rozpocząć nowy czat, kliknij odpowiedni przycisk.

W dowolnym momencie możesz również przerwać odpowiedź asystenta.


### Zatrzymywanie aplikacji

Gdy skończysz testować, uruchom komórkę pod nagłówkiem „Zatrzymywanie aplikacji”. To zwolni zasoby i poprawnie zakończy działanie czatu.

---

Nie krępuj się eksperymentować i zobaczyć, dokąd zaprowadzi Cię rozmowa z naszym Bielikiem!

### Modele z kontrolą dostępu

Na niektóre modele zostały nałożone zabezpieczenia w postaci kontrolowanego dostępu (gated models). Jeśli chciałbyś z tych modeli skorzystać, to należy podać token użytkownika, który można wygenerować na portalu HuggingFace: <br>
- tokeny użytkownika:
https://huggingface.co/settings/tokens
- więcej informacji o tokenach:
https://huggingface.co/docs/hub/security-tokens <br>

Odkomentuj i uruchom poniższy kod, aby zainstalować "huggingface_hub[cli]" i zalogować się.

In [ ]:
# !pip install -U "huggingface_hub[cli]" -q
# !huggingface-cli login

### Instalacja bibliotek

In [ ]:
import warnings
warnings.filterwarnings("ignore")

!pip install accelerate -q
!pip install -i https://pypi.org/simple/ bitsandbytes -q

!pip3 install piper-tts -q
!pip3 install ffmpeg-python -q
!pip3 install vosk -q

### Pobieranie głosów Piper

In [ ]:
# Create an application folder.
!mkdir voice_llm_chat
%cd voice_llm_chat

# Pobierzmy jakieś fajne głosy.

!wget -q -O pl_PL-mc_speech-medium.onnx https://huggingface.co/rhasspy/piper-voices/resolve/v1.0.0/pl/pl_PL/mc_speech/medium/pl_PL-mc_speech-medium.onnx?download=true
!wget -q -O pl_PL-mc_speech-medium.onnx.json https://huggingface.co/rhasspy/piper-voices/resolve/v1.0.0/pl/pl_PL/mc_speech/medium/pl_PL-mc_speech-medium.onnx.json?download=true

!wget -q -O pl_PL-darkman-medium.onnx https://huggingface.co/rhasspy/piper-voices/resolve/v1.0.0/pl/pl_PL/darkman/medium/pl_PL-darkman-medium.onnx?download=true
!wget -q -O pl_PL-darkman-medium.onnx.json https://huggingface.co/rhasspy/piper-voices/resolve/v1.0.0/pl/pl_PL/darkman/medium/pl_PL-darkman-medium.onnx.json?download=true

piper_voices = {"mc_speech": "pl_PL-mc_speech-medium.onnx", "darkman": "pl_PL-darkman-medium.onnx"}

### Import bibliotek

In [ ]:
import time
import sys
import json
from IPython.display import HTML, display
from vosk import Model, KaldiRecognizer
from base64 import b64decode
from google.colab import output
import ffmpeg
import threading
import ipywidgets as widgets
import IPython
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from piper.voice import PiperVoice

### Pobieranie skryptów czatu głosowego LLM

Kod źródłowy programu można sprawdzić na GitHubie pod [tym](https://github.com/artbert/VoiceChatLLM) linkiem.

In [ ]:
!wget -q https://raw.githubusercontent.com/artbert/VoiceChatLLM/refs/heads/main/utils/voice_llm_chat.py
!wget -q https://raw.githubusercontent.com/artbert/VoiceChatLLM/refs/heads/main/utils/voice_llm_chat_frontend.py

In [ ]:
from voice_llm_chat import VoiceLLMChatBackend
from voice_llm_chat_frontend import VoiceLLMChatFrontend_Colab

### Zmienne konfiguracyjne

In [ ]:
# Wybierz wiadomość systemową, która najlepiej pasuje do twoich potrzeb.
llm_model_system_message = "Jesteś pomocnym asystentem głosowym, który odpowiada w jednym lub dwóch krótkich zdaniach. Twoje odpowiedzi powinny unikać jakiegokolwiek formatowania tekstu."

# Możesz przetestować różne konfiguracje parametrów.
llm_model_temperature = 0.1
llm_model_max_tokens = 2048
llm_model_top_k = 100
llm_model_top_p = 1

### Ładowanie modeli

In [ ]:
# Wybieramy jeden z pobranych głosów i ładujemy syntezator mowy.
chosen_piper_voice = piper_voices["mc_speech"]
try:
    voice_model = PiperVoice.load(chosen_piper_voice)
except FileNotFoundError:
    print(f"""Error: Piper voice model file not found. Please ensure '{chosen_piper_voice}' is in the correct directory.""", file=sys.stderr)
    voice_model = None
except Exception as e:
    print(f"An unexpected error occurred while loading the Piper model: {e}", file=sys.stderr)
    voice_model = None

""" Ładujemy Vosk speech recognition model.
W przypadku języka polskiego mamy do dyspozycji tylko jeden mały model.
"""
sample_rate = 16000
try:
    speech_model = Model(model_name="vosk-model-small-pl-0.22")
except Exception as e:
    print(f"Error loading Vosk model: {e}. Please ensure the model is downloaded and accessible.", file=sys.stderr)
    speech_model = None
    speech_recognizer = None

if speech_model:
    try:
        speech_recognizer = KaldiRecognizer(speech_model, sample_rate)
        speech_recognizer.SetWords(True)
    except Exception as e:
        print(f"Error creating Vosk recognizer: {e}", file=sys.stderr)
        speech_recognizer = None

# Inicjalizacja modelu LLM i tokenizera.
# Możesz wybrać inny model. Pamiętaj, że może on mieć inne wymagania systemowe.
# llm_model_name = "speakleash/Bielik-11B-v2.3-Instruct"
llm_model_name = "speakleash/Bielik-11B-v2.6-Instruct"
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
tokenizer.pad_token = tokenizer.eos_token
# Za pomocą kwantyzacji ograniczymy znacznie użycie pamięci.
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16
            )

llm_model = AutoModelForCausalLM.from_pretrained(llm_model_name,
                                             dtype=torch.bfloat16,
                                             quantization_config=quantization_config,
                                             pad_token_id = tokenizer.pad_token_id,
                                             ).eval()

### Testowanie modeli

Zaleca się przetestowanie mikrofonu, modułu rozpoznawania mowy i syntezatora mowy. Aby to zrobić, uruchom poniższy kod. Kliknij przycisk **Record**, powiedz coś po polsku, a następnie zakończ klikając przycisk **Stop**. Przy pierwszym uruchomieniu zezwól przeglądarce na dostęp do mikrofonu i ponownie uruchom kod.

In [ ]:
from IPython.display import Javascript, Audio, display
from google.colab.output import eval_js

# Prosty kod Javascript, który umożliwi nagrywanie sygnału z mikrofonu i zakodowanie go w formacie tekstowym ASCII.
js = Javascript('''async function recordAudio() {
    const div = document.createElement('div');
    const startRecord = document.createElement('button');
    const stopRecord = document.createElement('button');

    startRecord.textContent = 'Record';
    stopRecord.textContent = 'Stop';

    document.body.appendChild(div);
    div.appendChild(startRecord);

    const stream = await navigator.mediaDevices.getUserMedia({audio:true});
    let audioRecorder = new MediaRecorder(stream);

    await new Promise((resolve) => startRecord.onclick = resolve);
    startRecord.replaceWith(stopRecord);
    audioRecorder.start();

    await new Promise((resolve) => stopRecord.onclick = resolve);
    audioRecorder.stop();
    let recData = await new Promise((resolve) => audioRecorder.ondataavailable = resolve);
    let arrBuff = await recData.data.arrayBuffer();
    stream.getAudioTracks()[0].stop();
    div.remove();

    let binaryString = '';
    let bytes = new Uint8Array(arrBuff);
    bytes.forEach((byte) => { binaryString += String.fromCharCode(byte) });

    const url = URL.createObjectURL(recData.data);
    const player = document.createElement('audio');
    player.controls = true;
    player.src = url;
    document.body.appendChild(player);

    return btoa(binaryString);
}
''')

# Dekodowanie formatu tekstowego do standardowego formatu binarnego.
def get_audio(data):
    if data is not None:
        try:
            binary = b64decode(data)
        except:
            print("Błąd podczas dekodowania. Sprawdź format danych wejściowych.")
        finally:
            process = (ffmpeg
            .input('pipe:0')
            .output('-', format='s16le', acodec='pcm_s16le', ac=1, ar='16k')
            .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
            )
            output, err = process.communicate(input=binary)
            return output

# Transkrypcja nagranego sygnału.
def transcribe(data):
    audio = get_audio(data)
    if audio is not None:
        speech_recognizer.AcceptWaveform(audio)
        result = json.loads(speech_recognizer.FinalResult())
        recognized_text = result['text']
        if recognized_text:
            recognized_text = recognized_text.capitalize() + "."

        return recognized_text
    else:
        return ""


display(js)
try:
    obj = eval_js('recordAudio({})')
    transcription = transcribe(obj)
    print(f"Rozpoznany tekst: {transcription}")

    for audio_chunk in voice_model.synthesize(transcription):
        display(Audio(audio_chunk.audio_int16_array, autoplay=True, rate=audio_chunk.sample_rate))

except Exception as e:
    print("Zezwól przeglądarce na dostęp do mikrofonu i uruchom ponownie kod.")
    print(f"Błąd: {e}")

### Frontend aplikacji

Komponenty interfejsu użytkownika i funkcje JavaScript, które zarządzają nagrywaniem głosu, i komunikacją z backendem Pythona w środowisku Colab zaimportujemy jako gotowiec. Jest to dokument HTML z osadzonym arkuszem stylów i skryptem Javascript. Jego zawartość można podejrzeć przez `print(llmChatFrontend)` lub bezpośrednio na GitHubie (link wyżej).

In [ ]:
voiceLLmFrontend = VoiceLLMChatFrontend_Colab(
    # Ustawiamy logo naszego asystenta...
    assistantAvatarSrc = "https://bielik.ai/wp-content/uploads/2024/08/Bielik_Secondary_Rabarbar-300x82.webp",
    # Dla użytkownika niech to będzie logo Golab
    userAvatarSrc = "https://colab.research.google.com/img/colab_favicon_256px.png"
    )

llmChatFrontend = voiceLLmFrontend.getDocument()
# # Można również podejrzeć zawartość dokumentu
# print(llmChatFrontend)

Pobrana wersja frontendu będzie korzystać z pięciu funkcji wywoływanych z poziomu kodu Javascript. Poniżej znajduje się klasa z implementacją tych metod.

### Klasa naszego programu

In [ ]:
class LLMChatApp:
    """Main application class for the voice-enabled LLM chat."""
    def __init__(self, llm_model, tokenizer, voice_model, speech_recognizer):
        """Initializes the LLMChatApp with required models and components."""
        self.output_lock = threading.Lock()
        # Output widget to display messages and recognized text
        self.app_output_widget = widgets.Output()

        # Preparing application's backend
        self.app = VoiceLLMChatBackend(llm_model, tokenizer, voice_model, speech_recognizer)
        # Passing parameters to the model.
        self.app.set_model_parameters(llm_model_temperature, llm_model_max_tokens, llm_model_top_k, llm_model_top_p, locale="pl")
        self.app.set_system_message(llm_model_system_message)

        # # If the application does not appear to function as intended, enable this flag.
        # self.app.should_print_logs = True

        self.initialized = self.app.initialized

    def new_chat(self):
        """Starts a new chat session."""
        self.app.start_new_chat()
        return IPython.display.JSON({"response": "new chat created"})

    def send_prompt(self, prompt):
        """Sends a user prompt to the LLM."""
        self.app.send_prompt(prompt)
        return IPython.display.JSON({"response": "New prompt sent"})

    def fetch_data(self):
        """Fetches completed data chunks from the LLM chat backend."""
        try:
            data = self.app.get_completed_data_chunk()
            if data is not None:
                display_sentence, encoded_audio = data
                """Many models return text formatted using Markdown notation, mainly headers and bolding.
                Here they need to be removed because they are not recognized by the downloaded version of the frontend."""
                display_sentence = display_sentence.replace("*", "").replace("_", "").replace("#", "")
                result = {
                    "resp": display_sentence,
                    "finish": "false"
                }
                if encoded_audio != "":
                    result["audio"] = encoded_audio

                return IPython.display.JSON(
                                result
                        )
            else:
                return IPython.display.JSON(
                {"resp": "", "finish": "true", "context": str(self.app.get_context_load())}
            )
        except Exception as e:
            print(f"Error in  fetch_data: {e}")

    def interrupt_response(self):
        """Interrupts the LLM's response generation."""
        self.app.interrupt_response()
        while self.app.is_model_working:
            time.sleep(0.1)
        response = self.app.get_last_response()
        return IPython.display.JSON(
            {
                "resp": response,
                "finish": "true",
                "context": str(self.app.get_context_load()),
            })

    def transcribe(self, data):
        """Transcribes audio data using the speech recognizer."""
        transcription = self.app.transcribe(data)
        return IPython.display.JSON({"result": transcription})

    def start_application(self):
        """Starts the main application logic."""
        self.app.start()

    def stop_application(self):
        """Stops the main application logic."""
        self.app.stop()

    def register_callbacks(self):
        """Registers the class methods as Colab output callbacks."""
        output.register_callback("notebook.new_chat", self.new_chat)
        output.register_callback("notebook.fetch_data", self.fetch_data)
        output.register_callback("notebook.transcribe", self.transcribe)
        output.register_callback("notebook.interrupt_response", self.interrupt_response)
        output.register_callback("notebook.send_prompt", self.send_prompt)


### Inicjalizacja aplikacji

In [ ]:
app_instance = LLMChatApp(llm_model, tokenizer, voice_model, speech_recognizer)

app_instance.register_callbacks()

### Uruchamianie aplikacji

Musisz zezwolić przeglądarce na korzystanie z mikrofonu, jeśli nie zrobiłeś tego jeszcze podczas testów. Przy pierwszym uruchomieniu aplikacji może być konieczne ponowne uruchomienie poniższego kodu. Pierwsza transkrypcja trwa nieco dłużej ze względu na inicjalizację modelu rozpoznawania mowy.

In [ ]:
if app_instance.initialized:
    app_instance.app_output_widget.outputs = []
    display(app_instance.app_output_widget)

    app_instance.start_application()

    app_instance.app_output_widget.append_display_data(HTML(llmChatFrontend))
else:
    print("initialization failed")
    # W razie problemów, ustaw flagę 'should_print_logs' na 'True', załaduj aplikację ponownie i sprawdź logi.

### Zatrzymywanie aplikacji

Aby prawidłowo zatrzymać aplikację i zwolnić zasoby, odkomentuj i uruchom poniższy kod.

In [ ]:
# app_instance.stop_application()

Całość konwersacji w postaci niesformatowanej listy wiadomości tekstowych można wyeksportować przez odwołanie do poniższego obiektu:

In [ ]:
# app_instance.app.chat_messages